In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow import keras

In [3]:
from keras.layers import Input, Lambda, Dense, Flatten

In [4]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [5]:
IMAGE_SIZE = [224, 224]


train_path = 'Datasets/train'
valid_path = 'Datasets/test'

In [6]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [7]:
for layer in vgg.layers:
    layer.trainable = False

In [8]:
folders = glob('Datasets/train/*')
x = Flatten()(vgg.output)

In [9]:
prediction = Dense(len(folders), activation='softmax')(x)
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
from keras.preprocessing.image import ImageDataGenerator

In [13]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)




# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('Datasets/train',
                                                 target_size = (224, 224),
                                                 batch_size = 50,
                                                 class_mode = 'categorical')




test_set = test_datagen.flow_from_directory('Datasets/test',
                                            target_size = (224, 224),
                                            batch_size = 50,
                                            class_mode = 'categorical')




Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [14]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)



Epoch 1/20
105/105 [==============================] - 301s 3s/step - loss: 0.2686 - accuracy: 0.8936 - val_loss: 0.2677 - val_accuracy: 0.9022
Epoch 2/20
105/105 [==============================] - 292s 3s/step - loss: 0.1152 - accuracy: 0.9567 - val_loss: 0.3215 - val_accuracy: 0.8942
Epoch 3/20
105/105 [==============================] - 291s 3s/step - loss: 0.0883 - accuracy: 0.9678 - val_loss: 0.2906 - val_accuracy: 0.9022
Epoch 4/20
105/105 [==============================] - 291s 3s/step - loss: 0.0840 - accuracy: 0.9695 - val_loss: 0.2926 - val_accuracy: 0.9103
Epoch 5/20
105/105 [==============================] - 290s 3s/step - loss: 0.0841 - accuracy: 0.9711 - val_loss: 0.3725 - val_accuracy: 0.8910
Epoch 6/20
105/105 [==============================] - 291s 3s/step - loss: 0.0768 - accuracy: 0.9712 - val_loss: 0.3314 - val_accuracy: 0.9038
Epoch 7/20
105/105 [==============================] - 294s 3s/step - loss: 0.0790 - accuracy: 0.9705 - val_loss: 0.4844 - val_accuracy: 0.8702

In [15]:
import tensorflow as tf
from keras.models import load_model

model.save('chest_xray.h5')